# 🎯 Conditional GAN (CGAN) 완전 정복

## 📚 학습 목표

1. Conditional GAN의 핵심 개념과 Vanilla GAN과의 차이점 이해
2. 조건 정보를 결합하는 다양한 방법 (Concatenation, Embedding, Projection) 비교
3. TensorFlow와 PyTorch로 CGAN 구현
4. Fashion MNIST로 조건부 이미지 생성 실습
5. CGAN의 평가 지표 (FID, Inception Score) 이해

---

# 🕐 1교시: DCGAN 복습 및 CGAN 소개

---

## 1.1 DCGAN 핵심 복습

### DCGAN의 5가지 설계 원칙

| 원칙 | 내용 | 효과 |
|------|------|------|
| 1️⃣ | Pooling → **Strided Conv** | 학습 가능한 다운샘플링 |
| 2️⃣ | **FC Layer 제거** (중간층) | 공간 구조 보존 |
| 3️⃣ | **Batch Normalization** | 학습 안정화 |
| 4️⃣ | **적절한 활성화 함수** | Gradient 흐름 개선 |
| 5️⃣ | **Adam (β1=0.5)** | GAN에 최적화 |

### DCGAN의 한계점

```
문제: 생성 제어 불가능

❌ DCGAN의 경우:
noise → G → 랜덤한 이미지
→ 어떤 이미지가 나올지 모름!

예시:
- 숫자 MNIST: 어떤 숫자가 나올지 모름
- CelebA: 어떤 스타일의 얼굴이 나올지 모름
- Fashion MNIST: 어떤 종류의 옷이 나올지 모름

💡 해결책: Conditional GAN (CGAN)
→ 조건 정보를 추가하여 원하는 이미지 생성!
```

---

## 1.2 Conditional GAN (CGAN) 소개

### CGAN의 핵심 아이디어

**"조건 정보(Condition)를 추가하여 생성을 제어하자!"**

```
GAN:    noise z → G → 랜덤 이미지

CGAN:   noise z + condition y → G → 조건에 맞는 이미지
        예: z + "숫자 7" → "7" 이미지 생성
```

### CGAN 논문 (2014)

**"Conditional Generative Adversarial Nets"**
- 저자: Mehdi Mirza, Simon Osindero
- 발표: arXiv 2014
- 핵심: GAN에 조건 정보를 추가한 첫 번째 논문

---

### GAN vs CGAN 비교

#### Vanilla GAN

```
Generator:
  입력: z (노이즈)
  출력: x (이미지)
  수식: G(z) = x

Discriminator:
  입력: x (이미지)
  출력: 진짜/가짜 확률
  수식: D(x) = [0, 1]

학습 목표:
  min_G max_D V(D,G) = E[log D(x)] + E[log(1 - D(G(z)))]
```

#### Conditional GAN

```
Generator:
  입력: z (노이즈) + y (조건)
  출력: x (조건에 맞는 이미지)
  수식: G(z, y) = x

Discriminator:
  입력: x (이미지) + y (조건)
  출력: 진짜/가짜 확률
  수식: D(x, y) = [0, 1]

학습 목표:
  min_G max_D V(D,G) = E[log D(x, y)] + E[log(1 - D(G(z, y), y))]
```

**핵심 차이:** 모든 곳에 조건 y가 추가됨!

---

### CGAN 아키텍처 개요

![](https://velog.velcdn.com/images%2Fwilko97%2Fpost%2F022d3c52-04ad-4e01-8307-f7c206a8ba1f%2Fimage.png)

---

## 1.3 조건 정보(Condition) 결합 방법

CGAN에서 조건 정보를 모델에 전달하는 방법은 여러 가지가 있습니다.

---

### 방법 1: One-Hot Encoding + Concatenation (원본 논문)

**가장 단순한 방법:**

```python
# 클래스 레이블을 One-Hot으로 변환
y = 7  # 숫자 7
y_onehot = [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]  # 10차원

# Generator
z = random_noise(100)  # 100차원
input_g = concat([z, y_onehot])  # 110차원
image = G(input_g)

# Discriminator
# 방법 A: 이미지를 flatten하여 결합
x_flat = flatten(image)  # 784차원 (28×28)
input_d = concat([x_flat, y_onehot])  # 794차원
output = D(input_d)
```

**장점:**
- ✅ 구현이 매우 간단
- ✅ 추가 파라미터 거의 없음

**단점:**
- ❌ 클래스 수가 많으면 비효율적 (1000개 클래스 → 1000차원 추가)
- ❌ 클래스 간 유사도를 표현할 수 없음
- ❌ Discriminator에서 공간 정보 손실 (flatten 필요)

---

### 방법 2: Embedding + Concatenation (일반적)

![](https://blog.kakaocdn.net/dna/cKUQKZ/btrQrrA21lu/AAAAAAAAAAAAAAAAAAAAAMMjchx7ANe_VJ5k7VGNM6iw4vGW9sEpB_CO9ZKjpWQD/img.png?credential=yqXZFxpELC7KVnFOS48ylbz2pIh7yKj8&expires=1764514799&allow_ip=&allow_referer=&signature=zMtRPd8IznSEWO2%2BwXJh%2FXfW9Wk%3D)

**Embedding을 사용한 개선 방법:**

```python
# Embedding Layer 사용
embedding_layer = Embedding(num_classes=10, embedding_dim=50)

# Generator
z = random_noise(100)  # 100차원
y_embed = embedding_layer(y)  # 10 → 50차원
input_g = concat([z, y_embed])  # 150차원
image = G(input_g)

# Discriminator
y_embed = embedding_layer(y)  # 50차원
y_map = Linear(y_embed)  # 50 → 784
y_map = reshape(y_map, [28, 28, 1])  # 공간 구조
input_d = concat([image, y_map], axis=channel)  # 28×28×2
output = D(input_d)
```

**Embedding의 역할:**
```
클래스 인덱스 → 학습 가능한 고차원 벡터

예시 (embedding_dim=3):
0 (T-shirt) → [0.2, -0.5,  0.8]
1 (Trouser) → [0.3,  0.1, -0.4]
7 (Sneaker) → [-0.6, 0.7,  0.2]

유사한 클래스는 유사한 벡터를 학습!
```

**장점:**
- ✅ 클래스 간 유사도 학습 가능
- ✅ 파라미터 효율적 (클래스 수 × 임베딩 차원)
- ✅ Discriminator에서 공간 정보 보존

**단점:**
- ❌ 추가 파라미터 필요 (Embedding layer)
- ❌ One-Hot보다 복잡

---

### 방법 3: Projection Discriminator (최신 기법)

**BigGAN, StyleGAN 등에서 사용:**

```python
# Discriminator에서 Projection 사용
# Generator는 동일 (Embedding + Concat)

# Discriminator
features = Conv_layers(image)  # (batch, feature_dim)
y_embed = Embedding(y)  # (batch, embedding_dim)

# Projection: 내적(dot product) 사용
output = Linear(features) + dot(features, y_embed)
#        └─ 무조건 판별      └─ 조건부 판별
```

> 이미지 특징과 라벨 임베딩의 유사도를 직접 계산해서,
>
> 라벨에 맞는 이미지인지 내적(dot product)으로 판별하는 강력한 조건 결합 방식.

**수식:**
$$
D(x, y) = \sigma(\phi(x)^T W + \phi(x)^T E(y))
$$

- $\phi(x)$: CNN 특징 벡터
- $W$: 학습 가능한 가중치
- $E(y)$: 임베딩 벡터

**장점:**
- ✅ 최고 성능 (BigGAN, StyleGAN2 등)
- ✅ 조건과 이미지의 상호작용 명확히 모델링
- ✅ 파라미터 효율적

**단점:**
- ❌ 구현이 복잡
- ❌ 작은 데이터셋에서는 오버헤드

---

### 세 가지 방법 비교 요약

| 방법 | 파라미터 | 성능 | 구현 난이도 | 사용처 |
|------|---------|------|------------|--------|
| **One-Hot** | 최소 | 보통 | 매우 쉬움 | 프로토타입, 클래스 수 적음 |
| **Embedding** | 중간 | 좋음 | 쉬움 | 일반적인 CGAN |
| **Projection** | 중간 | 최고 | 어려움 | 대규모 GAN (BigGAN 등) |

**본 강의에서는 Embedding + Concatenation 방식을 사용합니다.**
(가장 널리 사용되고, 성능과 구현의 균형이 좋음)

---

## 1.4 CGAN의 활용 사례

### 1. 조건부 이미지 생성

```
예시 1: MNIST
- 조건: 숫자 클래스 (0~9)
- 생성: "7"을 여러 스타일로 생성

예시 2: Fashion MNIST
- 조건: 옷 종류 (T-shirt, Sneaker, ...)
- 생성: 원하는 종류의 옷만 생성

예시 3: CelebA
- 조건: 속성 (안경, 금발, 웃음, ...)
- 생성: "안경 쓴 금발 여성" 생성
```

### 2. Image-to-Image Translation

```
Pix2Pix:
- 입력 이미지를 조건으로 사용
- 예: 스케치 → 사진, 낮 → 밤, 흑백 → 컬러

구조:
G(z, sketch) → photo
D(photo, sketch) → 진짜/가짜
```

### 3. Super Resolution

```
저해상도 이미지를 조건으로 사용:
G(z, low_res) → high_res
```

### 4. Text-to-Image

```
텍스트 설명을 조건으로 사용:
G(z, "a bird with red wings") → 이미지
```

---

## 1.5 CGAN Loss 함수 상세 분석

### Conditional GAN의 Min-Max 게임

$$
\min_G \max_D V(D, G) = \mathbb{E}_{x,y \sim p_{data}}[\log D(x, y)] + \mathbb{E}_{z \sim p_z, y \sim p_y}[\log(1 - D(G(z, y), y))]
$$

**Vanilla GAN과의 차이:**
- 모든 항에 조건 $y$가 추가됨
- $y$는 실제 데이터의 레이블 분포에서 샘플링

---

### Loss 계산 단계별 분석

#### Generator 학습

```python
# 1. 랜덤 노이즈와 랜덤 레이블 샘플링
z = sample_noise(batch_size, noise_dim)
y = sample_labels(batch_size, num_classes)

# 2. 가짜 이미지 생성
x_fake = G(z, y)

# 3. Discriminator로 판별
d_fake = D(x_fake, y)  # 조건 y도 함께 입력!

# 4. Generator Loss
# 목표: D(G(z, y), y)를 1에 가깝게
g_loss = BCE(d_fake, ones)
```

**핵심:** Generator는 "조건 y에 맞는" 가짜 이미지를 생성

#### Discriminator 학습

```python
# 1. 진짜 이미지와 레이블
x_real, y_real = sample_real_data()
d_real = D(x_real, y_real)
d_loss_real = BCE(d_real, ones)

# 2. 가짜 이미지 생성
z = sample_noise()
y_fake = sample_labels()  # 랜덤 레이블
x_fake = G(z, y_fake)
d_fake = D(x_fake, y_fake)
d_loss_fake = BCE(d_fake, zeros)

# 3. 총 Loss
d_loss = d_loss_real + d_loss_fake
```

**핵심:** Discriminator는 이미지와 레이블의 "페어링"이 맞는지 판별

---

### CGAN Loss의 직관적 이해

```
시나리오 1: 올바른 페어링
┌──────────────────────────────────────┐
│ 이미지: 스니커즈 사진                 │
│ 레이블: "Sneaker" (7)                │
│ → D의 판단: "진짜" (1)               │
└──────────────────────────────────────┘

시나리오 2: 잘못된 페어링
┌──────────────────────────────────────┐
│ 이미지: 스니커즈 사진                 │
│ 레이블: "T-shirt" (0)                │
│ → D의 판단: "가짜" (0)               │
└──────────────────────────────────────┘

시나리오 3: 가짜 이미지
┌──────────────────────────────────────┐
│ 이미지: G가 생성한 스니커즈           │
│ 레이블: "Sneaker" (7)                │
│ → D의 판단: "가짜" (0)               │
└──────────────────────────────────────┘
```

**Discriminator의 역할:**
1. 진짜 이미지 + 올바른 레이블 → 1
2. 진짜 이미지 + 잘못된 레이블 → 0
3. 가짜 이미지 + 임의 레이블 → 0

**Generator의 역할:**
- 조건 y에 "정확히 맞는" 이미지 생성
- D가 (G(z,y), y)를 1로 판별하도록

---

# 🕐 2교시: CGAN 구현 (TensorFlow)

---

## 2.1 환경 설정 및 데이터 준비

In [ ]:

# Colab용 한글 폰트 설치 및 설정 (한 번만 실행)
import os
import subprocess
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import tensorflow as tf
import numpy as np

# NanumGothic 설치 및 폰트 캐시 갱신
if not os.path.exists('/usr/share/fonts/truetype/nanum/NanumGothic.ttf'):
    subprocess.check_call(['apt-get', '-qq', '-y', 'install', 'fonts-nanum'])
    subprocess.check_call(['fc-cache', '-fv'])

# matplotlib에 폰트 등록
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
if os.path.exists(font_path):
    fm.fontManager.addfont(font_path)
    mpl.rcParams['font.family'] = 'NanumGothic'
    mpl.rcParams['axes.unicode_minus'] = False
else:
    print("NanumGothic 폰트를 찾을 수 없습니다. 설치가 정상적으로 되었는지 확인하세요.")

print(f"TensorFlow 버전: {tf.__version__}")
print("✅ 한글 폰트 설정 완료")

# GPU 확인
print(f"TensorFlow 버전: {tf.__version__}")
print(f"GPU 사용 가능: {tf.config.list_physical_devices('GPU')}")

# 랜덤 시드 설정
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
# 하이퍼파라미터 설정
IMG_SIZE = 28  # Fashion MNIST: 28×28
CHANNELS = 1  # Grayscale
NUM_CLASSES = 10  # 10개 클래스
BATCH_SIZE = 128
NOISE_DIM = 100
EMBEDDING_DIM = 50  # 클래스 임베딩 차원
EPOCHS = 50
LR = 0.0002
BETA1 = 0.5

# 클래스 이름 정의
CLASS_NAMES = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

print("✅ 하이퍼파라미터 설정 완료")
print(f"   - 이미지 크기: {IMG_SIZE}×{IMG_SIZE}×{CHANNELS}")
print(f"   - 클래스 수: {NUM_CLASSES}")
print(f"   - 임베딩 차원: {EMBEDDING_DIM}")

In [ ]:
# Fashion MNIST 데이터셋 로드
(x_train, y_train), (_, _) = tf.keras.datasets.fashion_mnist.load_data()

# 전처리: [0, 255] → [-1, 1]
x_train = x_train.astype("float32")
x_train = (x_train - 127.5) / 127.5
x_train = np.expand_dims(x_train, axis=-1)  # (60000, 28, 28, 1)

print(f"\n✅ 데이터셋 준비 완료")
print(f"   - 이미지 shape: {x_train.shape}")
print(f"   - 레이블 shape: {y_train.shape}")
print(f"   - 값 범위: [{x_train.min():.2f}, {x_train.max():.2f}]")

# Dataset 생성
train_dataset = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .shuffle(60000)
    .batch(BATCH_SIZE, drop_remainder=True)
)

In [ ]:
# 데이터 샘플 시각화
sample_images, sample_labels = next(iter(train_dataset))
print(f"배치 shape: {sample_images.shape}")

fig, axes = plt.subplots(2, 5, figsize=(15, 6))
for i in range(10):
    ax = axes[i // 5, i % 5]
    # 해당 클래스의 첫 번째 샘플 찾기
    idx = tf.where(sample_labels == i)[0][0]
    img = (sample_images[idx].numpy() + 1) / 2
    ax.imshow(img.squeeze(), cmap="gray")
    ax.set_title(f"{i}: {CLASS_NAMES[i]}", fontsize=10)
    ax.axis("off")

plt.suptitle("Fashion MNIST 클래스별 샘플", fontsize=14)
plt.tight_layout()
plt.show()

## 2.2 Conditional Generator 구현 (TensorFlow)

In [ ]:
def build_conditional_generator(noise_dim=100, num_classes=10, embedding_dim=50):
    """
    Conditional Generator 구현 (TensorFlow)

    구조:
        1. 노이즈 입력 (noise_dim)
        2. 레이블 입력 (정수) → Embedding (embedding_dim)
        3. Concatenate [noise, embedding]
        4. Dense + Reshape
        5. ConvTranspose layers

    Args:
        noise_dim: 노이즈 벡터 차원
        num_classes: 클래스 개수
        embedding_dim: 레이블 임베딩 차원

    Returns:
        tf.keras.Model
    """

    # ========================================
    # 입력층 정의
    # ========================================
    # 입력 1: 노이즈 벡터
    noise_input = layers.Input(shape=(noise_dim,), name="noise_input")
    # 예: (batch_size, 100)

    # 입력 2: 레이블 (정수)
    label_input = layers.Input(shape=(1,), dtype=tf.int32, name="label_input")
    # 예: (batch_size, 1) - 각 값은 0~9 사이의 정수

    # ========================================
    # 레이블 임베딩
    # ========================================
    # Embedding Layer: 정수 → 고차원 벡터
    # input_dim: 클래스 개수 (10)
    # output_dim: 임베딩 차원 (50)
    label_embedding = layers.Embedding(
        input_dim=num_classes, output_dim=embedding_dim, name="label_embedding"
    )(label_input)
    # 입력: (batch_size, 1) - 정수
    # 출력: (batch_size, 1, embedding_dim)
    # 예: (128, 1, 50)

    # Flatten: (batch_size, 1, embedding_dim) → (batch_size, embedding_dim)
    label_embedding = layers.Flatten()(label_embedding)
    # 예: (128, 1, 50) → (128, 50)

    # ========================================
    # 노이즈와 임베딩 결합
    # ========================================
    # Concatenate: [noise, label_embedding]
    # noise: (batch_size, 100)
    # label_embedding: (batch_size, 50)
    # → gen_input: (batch_size, 150)
    gen_input = layers.Concatenate()([noise_input, label_embedding])

    # ========================================
    # Generator 메인 네트워크
    # ========================================
    # Dense: 150 → 7×7×256 = 12,544
    x = layers.Dense(7 * 7 * 256, use_bias=False)(gen_input)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Reshape: (batch_size, 12544) → (batch_size, 7, 7, 256)
    x = layers.Reshape((7, 7, 256))(x)

    # ConvTranspose 1: 7×7 → 14×14
    x = layers.Conv2DTranspose(128, 4, 2, "same", use_bias=False)(x)
    # 출력: (batch_size, 14, 14, 128)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # ConvTranspose 2 (출력층): 14×14 → 28×28
    x = layers.Conv2DTranspose(1, 4, 2, "same", use_bias=False, activation="tanh")(x)
    # 출력: (batch_size, 28, 28, 1)

    # ========================================
    # 모델 정의
    # ========================================
    model = tf.keras.Model(
        inputs=[noise_input, label_input], outputs=x, name="Conditional_Generator"
    )

    return model


# Generator 생성
generator = build_conditional_generator(NOISE_DIM, NUM_CLASSES, EMBEDDING_DIM)
generator.summary()

In [ ]:
# Generator 테스트
print("\n🧪 Conditional Generator 테스트")
print("=" * 60)

# 테스트 입력
test_noise = tf.random.normal([4, NOISE_DIM])
test_labels = tf.constant([[0], [3], [7], [9]])  # T-shirt, Dress, Sneaker, Ankle boot

print(f"입력 노이즈 shape: {test_noise.shape}")
print(f"입력 레이블: {test_labels.numpy().flatten()}")
print(f"레이블 이름: {[CLASS_NAMES[i] for i in test_labels.numpy().flatten()]}")

# Generator 통과
test_output = generator([test_noise, test_labels], training=False)
print(f"\n출력 이미지 shape: {test_output.shape}")
print(
    f"출력 값 범위: [{test_output.numpy().min():.3f}, {test_output.numpy().max():.3f}]"
)

# 시각화
fig, axes = plt.subplots(1, 4, figsize=(12, 3))
for i, ax in enumerate(axes):
    img = (test_output[i].numpy() + 1) / 2
    ax.imshow(img.squeeze(), cmap="gray")
    ax.set_title(f"{CLASS_NAMES[test_labels[i, 0]]}", fontsize=10)
    ax.axis("off")
plt.suptitle("Generator 초기 출력 (학습 전)", fontsize=14)
plt.tight_layout()
plt.show()

## 2.3 Conditional Discriminator 구현 (TensorFlow)

In [ ]:
def build_conditional_discriminator(num_classes=10, embedding_dim=50):
    """
    Conditional Discriminator 구현 (TensorFlow)

    구조:
        1. 이미지 입력 (28×28×1)
        2. 레이블 입력 (정수) → Embedding → Linear → Reshape → 28×28×1 맵
        3. Concatenate [image, label_map] → 28×28×2
        4. Conv layers
        5. Dense → 출력 (진짜/가짜)

    Args:
        num_classes: 클래스 개수
        embedding_dim: 레이블 임베딩 차원

    Returns:
        tf.keras.Model
    """

    # ========================================
    # 입력층 정의
    # ========================================
    # 입력 1: 이미지
    image_input = layers.Input(shape=(28, 28, 1), name="image_input")
    # 예: (batch_size, 28, 28, 1)

    # 입력 2: 레이블 (정수)
    label_input = layers.Input(shape=(1,), dtype=tf.int32, name="label_input")
    # 예: (batch_size, 1)

    # ========================================
    # 레이블 임베딩 및 맵 생성
    # ========================================
    # Step 1: Embedding
    label_embedding = layers.Embedding(input_dim=num_classes, output_dim=embedding_dim)(
        label_input
    )
    # 출력: (batch_size, 1, embedding_dim)

    label_embedding = layers.Flatten()(label_embedding)
    # 출력: (batch_size, embedding_dim)
    # 예: (128, 50)

    # Step 2: Linear로 28×28 크기로 변환
    label_map = layers.Dense(28 * 28)(label_embedding)
    # 출력: (batch_size, 784)

    # Step 3: Reshape to 28×28×1
    label_map = layers.Reshape((28, 28, 1))(label_map)
    # 출력: (batch_size, 28, 28, 1)

    # ========================================
    # 이미지와 레이블 맵 결합
    # ========================================
    # Concatenate: [image, label_map]
    # image: (batch_size, 28, 28, 1)
    # label_map: (batch_size, 28, 28, 1)
    # → disc_input: (batch_size, 28, 28, 2)
    disc_input = layers.Concatenate(axis=-1)([image_input, label_map])

    # ========================================
    # Discriminator 메인 네트워크
    # ========================================
    # Conv 1: 28×28 → 14×14
    x = layers.Conv2D(64, 4, 2, "same")(disc_input)
    # 출력: (batch_size, 14, 14, 64)
    x = layers.LeakyReLU(0.2)(x)

    # Conv 2: 14×14 → 7×7
    x = layers.Conv2D(128, 4, 2, "same", use_bias=False)(x)
    # 출력: (batch_size, 7, 7, 128)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)

    # Flatten + Dense
    x = layers.Flatten()(x)
    # 출력: (batch_size, 6272)

    x = layers.Dense(1)(x)
    # 출력: (batch_size, 1) - Logit 값 (Sigmoid 없음)

    # ========================================
    # 모델 정의
    # ========================================
    model = tf.keras.Model(
        inputs=[image_input, label_input], outputs=x, name="Conditional_Discriminator"
    )

    return model


# Discriminator 생성
discriminator = build_conditional_discriminator(NUM_CLASSES, EMBEDDING_DIM)
discriminator.summary()

In [ ]:
# Discriminator 테스트
print("\n🧪 Conditional Discriminator 테스트")
print("=" * 60)

# 진짜 이미지
real_sample = sample_images[:4]
real_labels = tf.reshape(sample_labels[:4], [-1, 1])
print(f"진짜 이미지 shape: {real_sample.shape}")
print(f"진짜 레이블: {real_labels.numpy().flatten()}")

# 가짜 이미지
fake_sample = generator([test_noise, test_labels], training=False)
print(f"가짜 이미지 shape: {fake_sample.shape}")
print(f"가짜 레이블: {test_labels.numpy().flatten()}")

# Discriminator 통과
real_output = discriminator([real_sample, real_labels], training=False)
fake_output = discriminator([fake_sample, test_labels], training=False)

print(f"\n진짜 이미지 판별 결과: {real_output.numpy().flatten()}")
print(f"가짜 이미지 판별 결과: {fake_output.numpy().flatten()}")
print("=" * 60)

## 2.4 손실 함수 및 Optimizer 설정

In [ ]:
# BCEWithLogitsLoss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Optimizer
optimizer_g = tf.keras.optimizers.Adam(learning_rate=LR, beta_1=BETA1, beta_2=0.999)
optimizer_d = tf.keras.optimizers.Adam(learning_rate=LR, beta_1=BETA1, beta_2=0.999)

print("✅ 손실 함수 및 Optimizer 설정 완료")

## 2.5 학습 Step 함수 구현

In [ ]:
@tf.function
def train_step(real_images, real_labels):
    """
    CGAN 한 스텝 학습 (TensorFlow)

    Args:
        real_images: 진짜 이미지 (batch_size, 28, 28, 1)
        real_labels: 진짜 레이블 (batch_size,)

    Returns:
        d_loss, g_loss
    """
    batch_size = tf.shape(real_images)[0]

    # 레이블 reshape: (batch_size,) → (batch_size, 1)
    real_labels = tf.reshape(real_labels, [-1, 1])

    # ========================================
    # Discriminator 학습
    # ========================================
    with tf.GradientTape() as disc_tape:
        # 1. 노이즈 및 랜덤 레이블 생성
        noise = tf.random.normal([batch_size, NOISE_DIM])
        fake_labels = tf.random.uniform(
            [batch_size, 1], minval=0, maxval=NUM_CLASSES, dtype=tf.int32
        )

        # 2. 가짜 이미지 생성
        fake_images = generator([noise, fake_labels], training=True)

        # 3. Discriminator 판별
        real_output = discriminator([real_images, real_labels], training=True)
        fake_output = discriminator([fake_images, fake_labels], training=True)

        # 4. Loss 계산
        d_loss_real = cross_entropy(tf.ones_like(real_output), real_output)
        d_loss_fake = cross_entropy(tf.zeros_like(fake_output), fake_output)
        d_loss = d_loss_real + d_loss_fake

    # Discriminator gradient 계산 및 적용
    gradients_of_discriminator = disc_tape.gradient(
        d_loss, discriminator.trainable_variables
    )
    optimizer_d.apply_gradients(
        zip(gradients_of_discriminator, discriminator.trainable_variables)
    )

    # ========================================
    # Generator 학습
    # ========================================
    noise = tf.random.normal([batch_size, NOISE_DIM])
    fake_labels = tf.random.uniform(
        [batch_size, 1], minval=0, maxval=NUM_CLASSES, dtype=tf.int32
    )

    with tf.GradientTape() as gen_tape:
        fake_images = generator([noise, fake_labels], training=True)
        fake_output = discriminator([fake_images, fake_labels], training=True)

        # Generator Loss: 가짜를 진짜(1)로
        g_loss = cross_entropy(tf.ones_like(fake_output), fake_output)

    # Generator gradient 계산 및 적용
    gradients_of_generator = gen_tape.gradient(g_loss, generator.trainable_variables)
    optimizer_g.apply_gradients(
        zip(gradients_of_generator, generator.trainable_variables)
    )

    return d_loss, g_loss


print("✅ train_step 함수 정의 완료")

## 2.6 이미지 생성 함수

In [ ]:
# 고정 노이즈 및 레이블 (학습 과정 비교용)
fixed_noise = tf.random.normal([NUM_CLASSES, NOISE_DIM])
fixed_labels = tf.reshape(tf.range(NUM_CLASSES), [-1, 1])


def generate_and_save_images(epoch, generator, noise, labels):
    """이미지 생성 및 시각화"""
    predictions = generator([noise, labels], training=False)

    fig, axes = plt.subplots(2, 5, figsize=(12, 5))
    for i, ax in enumerate(axes.flat):
        img = (predictions[i].numpy() + 1) / 2
        ax.imshow(img.squeeze(), cmap="gray")
        ax.set_title(f"{CLASS_NAMES[labels[i, 0]]}", fontsize=10)
        ax.axis("off")

    plt.suptitle(f"Epoch {epoch}", fontsize=16)
    plt.tight_layout()
    plt.show()


# 초기 이미지
print("🎨 학습 전 Generator 출력")
generate_and_save_images(0, generator, fixed_noise, fixed_labels)

## 2.7 전체 학습 실행

In [ ]:
# 학습 실행
print("=" * 60)
print("🔥 CGAN 학습 시작 (TensorFlow)")
print("=" * 60)

history_tf = {"d_loss": [], "g_loss": []}

for epoch in range(1, EPOCHS + 1):
    print(f"\n🔥 Epoch {epoch}/{EPOCHS}")

    epoch_d_loss = []
    epoch_g_loss = []

    for batch_idx, (real_batch, real_labels_batch) in enumerate(train_dataset):
        d_loss, g_loss = train_step(real_batch, real_labels_batch)

        epoch_d_loss.append(d_loss.numpy())
        epoch_g_loss.append(g_loss.numpy())

        if (batch_idx + 1) % 100 == 0:
            print(
                f"  Batch {batch_idx + 1} - D Loss: {d_loss:.4f}, G Loss: {g_loss:.4f}"
            )

    avg_d_loss = np.mean(epoch_d_loss)
    avg_g_loss = np.mean(epoch_g_loss)
    history_tf["d_loss"].append(avg_d_loss)
    history_tf["g_loss"].append(avg_g_loss)

    print(f"📊 Epoch {epoch} 평균 - D Loss: {avg_d_loss:.4f}, G Loss: {avg_g_loss:.4f}")

    if epoch % 10 == 0:
        generate_and_save_images(epoch, generator, fixed_noise, fixed_labels)

print("\n✅ 학습 완료!")

In [ ]:
# 학습 곡선
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history_tf["d_loss"], label="D Loss", color="blue")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Discriminator Loss")
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history_tf["g_loss"], label="G Loss", color="red")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Generator Loss")
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2.8 조건부 생성 예제

In [ ]:
# 특정 클래스 생성
def generate_specific_class_tf(class_idx, num_samples=9):
    """특정 클래스의 이미지 생성"""
    noise = tf.random.normal([num_samples, NOISE_DIM])
    labels = tf.fill([num_samples, 1], class_idx)

    images = generator([noise, labels], training=False)

    rows = int(np.ceil(np.sqrt(num_samples)))
    cols = int(np.ceil(num_samples / rows))

    fig, axes = plt.subplots(rows, cols, figsize=(cols * 2, rows * 2))
    axes = axes.flatten() if num_samples > 1 else [axes]

    for i in range(num_samples):
        img = (images[i].numpy() + 1) / 2
        axes[i].imshow(img.squeeze(), cmap="gray")
        axes[i].axis("off")

    for i in range(num_samples, len(axes)):
        axes[i].axis("off")

    plt.suptitle(f"{CLASS_NAMES[class_idx]} × {num_samples}", fontsize=14)
    plt.tight_layout()
    plt.show()


# 스니커즈 16개 생성
print("👟 스니커즈 16개 생성")
generate_specific_class_tf(class_idx=7, num_samples=16)

---
# 🕐 3교시: CGAN 구현 (PyTorch)

---

## 3.1 환경 설정

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"PyTorch 버전: {torch.__version__}")
print(f"사용 디바이스: {device}")

# 랜덤 시드
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

## 3.2 데이터 로드

In [ ]:
# 데이터 전처리
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
)

# Fashion MNIST 로드
train_dataset_pt = datasets.FashionMNIST(
    root="./data", train=True, download=True, transform=transform
)

train_dataloader_pt = DataLoader(
    train_dataset_pt, batch_size=BATCH_SIZE, shuffle=True, drop_last=True
)

print(f"데이터셋 크기: {len(train_dataset_pt)}")
print(f"배치 수: {len(train_dataloader_pt)}")

## 3.3 Conditional Generator 구현 (PyTorch)

In [ ]:
class ConditionalGenerator(nn.Module):
    """
    Conditional Generator (PyTorch)

    구조:
        1. 레이블 임베딩
        2. 노이즈 + 임베딩 결합
        3. Dense + Reshape
        4. ConvTranspose layers
    """

    def __init__(self, noise_dim=100, num_classes=10, embedding_dim=50):
        super(ConditionalGenerator, self).__init__()

        # 레이블 임베딩
        self.label_embedding = nn.Embedding(
            num_embeddings=num_classes, embedding_dim=embedding_dim
        )

        # Dense layer
        input_dim = noise_dim + embedding_dim
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 7 * 7 * 256),
            nn.BatchNorm1d(7 * 7 * 256),
            nn.ReLU(True),
        )

        # ConvTranspose layers
        self.main = nn.Sequential(
            # 7×7 → 14×14
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            # 14×14 → 28×28
            nn.ConvTranspose2d(128, 1, 4, 2, 1, bias=False),
            nn.Tanh(),
        )

    def forward(self, z, labels):
        """
        Args:
            z: (batch_size, noise_dim)
            labels: (batch_size,) - LongTensor
        """
        # 레이블 임베딩
        label_embed = self.label_embedding(labels)  # (batch, embedding_dim)

        # 노이즈와 결합
        gen_input = torch.cat(
            [z, label_embed], dim=1
        )  # (batch, noise_dim+embedding_dim)

        # Dense
        out = self.fc(gen_input)  # (batch, 12544)

        # Reshape
        out = out.view(-1, 256, 7, 7)  # (batch, 256, 7, 7)

        # ConvTranspose
        img = self.main(out)  # (batch, 1, 28, 28)

        return img


# 가중치 초기화
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
    elif classname.find("Embedding") != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)


# Generator 생성
generator_pt = ConditionalGenerator(NOISE_DIM, NUM_CLASSES, EMBEDDING_DIM).to(device)
generator_pt.apply(weights_init)

print("📊 Conditional Generator (PyTorch)")
print(generator_pt)

## 3.4 Conditional Discriminator 구현 (PyTorch)

In [ ]:
class ConditionalDiscriminator(nn.Module):
    """
    Conditional Discriminator (PyTorch)

    구조:
        1. 레이블 → 임베딩 → Linear → 28×28 맵
        2. 이미지 + 레이블 맵 결합
        3. Conv layers
    """

    def __init__(self, num_classes=10, embedding_dim=50):
        super(ConditionalDiscriminator, self).__init__()

        self.img_size = 28

        # 레이블 임베딩
        self.label_embedding = nn.Embedding(num_classes, embedding_dim)

        # 임베딩 → 28×28 맵
        self.label_fc = nn.Linear(embedding_dim, self.img_size * self.img_size)

        # Conv layers (입력 채널: 2)
        self.main = nn.Sequential(
            # 28×28 → 14×14
            nn.Conv2d(2, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # 14×14 → 7×7
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
        )

        # Dense
        self.fc = nn.Sequential(nn.Flatten(), nn.Linear(128 * 7 * 7, 1))

    def forward(self, img, labels):
        """
        Args:
            img: (batch_size, 1, 28, 28)
            labels: (batch_size,) - LongTensor
        """
        batch_size = img.size(0)

        # 레이블 임베딩
        label_embed = self.label_embedding(labels)  # (batch, embedding_dim)

        # 임베딩 → 784
        label_map = self.label_fc(label_embed)  # (batch, 784)

        # Reshape to 28×28×1
        label_map = label_map.view(batch_size, 1, self.img_size, self.img_size)

        # 이미지와 결합
        disc_input = torch.cat([img, label_map], dim=1)  # (batch, 2, 28, 28)

        # Conv layers
        features = self.main(disc_input)  # (batch, 128, 7, 7)

        # Dense
        output = self.fc(features).view(-1)  # (batch,)

        return output


# Discriminator 생성
discriminator_pt = ConditionalDiscriminator(NUM_CLASSES, EMBEDDING_DIM).to(device)
discriminator_pt.apply(weights_init)

print("📊 Conditional Discriminator (PyTorch)")
print(discriminator_pt)

## 3.5 손실 함수 및 Optimizer

In [ ]:
# Loss
criterion_pt = nn.BCEWithLogitsLoss()

# Optimizer
optimizer_g_pt = optim.Adam(generator_pt.parameters(), lr=LR, betas=(BETA1, 0.999))
optimizer_d_pt = optim.Adam(discriminator_pt.parameters(), lr=LR, betas=(BETA1, 0.999))

print("✅ 손실 함수 및 Optimizer 설정 완료")

## 3.6 학습 Step 함수

In [ ]:
def train_step_pt(real_images, real_labels):
    """CGAN 한 스텝 학습 (PyTorch)"""
    batch_size = real_images.size(0)
    real_images = real_images.to(device)
    real_labels = real_labels.to(device)

    # 타겟
    real_target = torch.ones(batch_size, device=device)
    fake_target = torch.zeros(batch_size, device=device)

    # ========================================
    # Discriminator 학습
    # ========================================
    discriminator_pt.zero_grad()

    # 진짜 이미지
    real_output = discriminator_pt(real_images, real_labels)
    d_loss_real = criterion_pt(real_output, real_target)

    # 가짜 이미지
    noise = torch.randn(batch_size, NOISE_DIM, device=device)
    fake_labels = torch.randint(0, NUM_CLASSES, (batch_size,), device=device)
    fake_images = generator_pt(noise, fake_labels)
    fake_output = discriminator_pt(fake_images.detach(), fake_labels)
    d_loss_fake = criterion_pt(fake_output, fake_target)

    # D Loss
    d_loss = d_loss_real + d_loss_fake
    d_loss.backward()
    optimizer_d_pt.step()

    # ========================================
    # Generator 학습
    # ========================================
    generator_pt.zero_grad()

    noise = torch.randn(batch_size, NOISE_DIM, device=device)
    fake_labels = torch.randint(0, NUM_CLASSES, (batch_size,), device=device)
    fake_images = generator_pt(noise, fake_labels)
    fake_output = discriminator_pt(fake_images, fake_labels)

    # G Loss
    g_loss = criterion_pt(fake_output, real_target)
    g_loss.backward()
    optimizer_g_pt.step()

    return d_loss.item(), g_loss.item()


print("✅ train_step 함수 정의 완료")

## 3.7 전체 학습 실행

In [ ]:
# 고정 노이즈
fixed_noise_pt = torch.randn(NUM_CLASSES, NOISE_DIM, device=device)
fixed_labels_pt = torch.arange(NUM_CLASSES, device=device)


def generate_and_save_images_pt(epoch, generator, noise, labels):
    """이미지 생성 (PyTorch)"""
    generator.eval()

    with torch.no_grad():
        fake_images = generator(noise, labels).cpu()

    fig, axes = plt.subplots(2, 5, figsize=(12, 5))
    for i, ax in enumerate(axes.flat):
        img = fake_images[i].squeeze().numpy()
        img = (img + 1) / 2
        ax.imshow(img, cmap="gray")
        ax.set_title(f"{CLASS_NAMES[labels[i]]}", fontsize=10)
        ax.axis("off")

    plt.suptitle(f"Epoch {epoch}", fontsize=16)
    plt.tight_layout()
    plt.show()

    generator.train()


# 초기 이미지
print("🎨 학습 전 Generator 출력 (PyTorch)")
generate_and_save_images_pt(0, generator_pt, fixed_noise_pt, fixed_labels_pt)

In [ ]:
# 학습 실행
print("=" * 60)
print("🔥 CGAN 학습 시작 (PyTorch)")
print("=" * 60)

history_pt = {"d_loss": [], "g_loss": []}

for epoch in range(1, EPOCHS + 1):
    print(f"\n🔥 Epoch {epoch}/{EPOCHS}")

    epoch_d_loss = []
    epoch_g_loss = []

    for batch_idx, (real_batch, real_labels_batch) in enumerate(train_dataloader_pt):
        d_loss, g_loss = train_step_pt(real_batch, real_labels_batch)

        epoch_d_loss.append(d_loss)
        epoch_g_loss.append(g_loss)

        if (batch_idx + 1) % 100 == 0:
            print(
                f"  Batch {batch_idx + 1} - D Loss: {d_loss:.4f}, G Loss: {g_loss:.4f}"
            )

    avg_d_loss = np.mean(epoch_d_loss)
    avg_g_loss = np.mean(epoch_g_loss)
    history_pt["d_loss"].append(avg_d_loss)
    history_pt["g_loss"].append(avg_g_loss)

    print(f"📊 Epoch {epoch} 평균 - D Loss: {avg_d_loss:.4f}, G Loss: {avg_g_loss:.4f}")

    if epoch % 10 == 0:
        generate_and_save_images_pt(
            epoch, generator_pt, fixed_noise_pt, fixed_labels_pt
        )

print("\n✅ 학습 완료!")

In [ ]:
# 학습 곡선
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history_pt["d_loss"], label="D Loss", color="blue")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Discriminator Loss (PyTorch)")
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(history_pt["g_loss"], label="G Loss", color="red")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Generator Loss (PyTorch)")
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
# 📝 정리 및 다음 시간 예고

---

## 🎯 Day 3 핵심 요약

### 1. CGAN의 핵심

- **조건 정보 추가:** GAN에 조건 y를 추가하여 생성 제어
- **Generator:** G(z, y) → 조건에 맞는 이미지
- **Discriminator:** D(x, y) → 이미지와 조건의 페어링 판별

### 2. 조건 결합 방식

| 방식 | 특징 | 적합한 경우 |
|------|------|-----------|
| **One-Hot** | 간단, 파라미터 적음 | 프로토타입, 클래스 적음 |
| **Embedding** | 유사도 학습 가능 | 일반적인 CGAN |
| **Projection** | 최고 성능 | 대규모, 고품질 필요 |


### 3. CGAN의 장점

- ✅ 생성 제어 가능 (원하는 클래스 지정)
- ✅ 학습 안정성 향상 (조건 정보가 가이드 역할)
- ✅ 다양한 응용 (Image-to-Image Translation 등)